In [142]:
import numpy as np 
import pandas as pd

In [143]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [144]:
train.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [145]:
Y_train = train.iloc[:,-1] 
Y_train

0       1
1       1
2       1
3       1
4       1
       ..
7608    1
7609    1
7610    1
7611    1
7612    1
Name: target, Length: 7613, dtype: int64

In [146]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
from nltk.stem import WordNetLemmatizer

In [147]:
wl = WordNetLemmatizer()

In [148]:
corpus=[]
text=[]
for comment in train.loc[:,'text']:
    comment = re.sub(r'(.)1+', r'1', comment)
    comment = re.sub('((http:\.+)|(www.[^s]+))','',comment) 
    comment=re.sub('[^a-zA-Z]',' ',comment)
    comment=comment.lower()
    comment=comment.split()
    text=text+comment
    comment=[wnl.lemmatize(word) for word in comment if word not in set(stopwords.words('english'))]
    comment=' '.join(comment)
    corpus.append(comment)

In [149]:
corpus[0]

'deed reason earthquake may allah forgive u'

In [150]:
from nltk.probability import FreqDist
fd = FreqDist()
for word in text:
    fd[word]+=1 

In [151]:
fd

FreqDist({'t': 5315, 'co': 4741, 'http': 4309, 'the': 3286, 'a': 2307, 'in': 1991, 'to': 1952, 'i': 1876, 'of': 1833, 'and': 1426, ...})

In [152]:
fd_values = list(fd.values())
fd_values.sort(reverse=True)

In [153]:
len(fd_values)

22027

In [154]:
s=0
s1=sum(fqd_values)
iteration =1 
l=[]
for i in range(0,20000,200): 
    s=s+sum(fqd_values[i:i+200])
    print(f'for top {i+200} words {(s/s1)*100}% of data')
    l.append((s/s1)*100) 
    iteration+=1 

for top 200 words 48.9223372165985% of data
for top 400 words 56.280381321015746% of data
for top 600 words 61.5299711968067% of data
for top 800 words 65.4520438171413% of data
for top 1000 words 68.36411469870045% of data
for top 1200 words 70.65400636543012% of data
for top 1400 words 72.53037187114919% of data
for top 1600 words 74.09300180681448% of data
for top 1800 words 75.39859537183335% of data
for top 2000 words 76.54845513581348% of data
for top 2200 words 77.56828473581953% of data
for top 2400 words 78.47547192633641% of data
for top 2600 words 79.28135654724555% of data
for top 2800 words 80.03734587267628% of data
for top 3000 words 80.77519145429667% of data
for top 3200 words 81.37998291464125% of data
for top 3400 words 81.98477437498582% of data
for top 3600 words 82.5895658353304% of data
for top 3800 words 83.09532269404356% of data
for top 4000 words 83.548916289302% of data
for top 4200 words 84.00250988456042% of data
for top 4400 words 84.45610347981886% of da

In [155]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=4000)
train = tfidf.fit_transform(corpus).toarray()

In [156]:
train.shape

(7613, 4000)

In [158]:
tweets1 = []
for st in test.loc[:,'text']:  
    st = re.sub(r'(.)1+', r'1', st)
    st = re.sub('((http:\.+)|(www.[^s]+))','',st) 
    st = re.sub('[^a-zA-Z]',' ',st) 
    st = st.split(' ')
    st = [wl.lemmatize(i) for i in st if i not in set(stopwords.words('english'))] 
    st = ' '.join(st) 
    tweets1.append(st)  
tweets1[0:5] 

['Just happened terrible car crash',
 'Heard  earthquake different city  stay safe everyone ',
 'forest fire spot pond  goose fleeing across street  I cannot save',
 'Apocalypse lighting   Spokane  wildfire',
 'Typhoon Soudelor kill    China Taiwan']

In [159]:
test = tfidf.transform(tweets1) 
test = test.toarray()

In [160]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=2000)

In [162]:
from sklearn.model_selection import cross_val_score
cross_val_score(lr,train,Y_train).mean()


0.7064294915543791

In [168]:
lr.fit(train,Y_train)
Y_prediction = lr.predict(test)  

In [169]:
result = pd.read_csv("result.csv")

In [170]:
result['target']=Y_prediction

In [171]:
result.head(5)

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
